In [24]:
import pandas as pd
import numpy as np

Load Data

# NHANES II

In [38]:
nhanesII = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanesII_frs_atp3.dta")
nhanesII.head()
test=nhanesII.head()

See Ranges

In [39]:
print(nhanesII[["AGEYR_AT_INT","SEX","lbdhdd","lbxtc","treated","asbp","curr_smq"]].min())
print(nhanesII[["AGEYR_AT_INT","SEX","lbdhdd","lbxtc","treated","asbp","curr_smq"]].max())
nhanesII.curr_smq.value_counts()
nhanesII["curr_smq"][1]==1

AGEYR_AT_INT    45.0
SEX              1.0
lbdhdd          15.0
lbxtc           80.0
treated          0.0
asbp            70.0
curr_smq         0.0
dtype: float64
AGEYR_AT_INT     74.0
SEX               2.0
lbdhdd          187.0
lbxtc           644.0
treated           1.0
asbp            300.0
curr_smq          1.0
dtype: float64


False

Create `treated` variable

Define `check` functions for females

In [5]:
def ftchol_pts(totchol,age):
    if totchol < 160:
        return(0)
    if totchol in range(160,199+1):
        if age in range(20,39+1): return(4)
        if age in range(40,49+1): return(3) 
        if age in range(50,59+1): return(2) 
        if age in range(60,69+1): return(1) 
        if age >= 70: return(1) 
    if totchol in range(200,239+1):
        if age in range(20,39+1): return(8) 
        if age in range(40,49+1): return(6) 
        if age in range(50,59+1): return(4) 
        if age in range(60,69+1): return(2) 
        if age >= 70: return(1) 
    if totchol in range(240,279+1):
        if age in range(20,39+1): return(11) 
        if age in range(40,49+1): return(8) 
        if age in range(50,59+1): return(5) 
        if age in range(60,69+1): return(3) 
        if age >= 70: return(2) 
    if totchol >= 280:
        if age in range(20,39+1): return(13) 
        if age in range(40,49+1): return(10)
        if age in range(50,59+1): return(7)
        if age in range(60,69+1): return(4) 
        if age >= 70: return(2) 
    
def fsmoke_pts(curr_smq,age):
    if curr_smq == 0:
        return(0)
    if curr_smq == 1:
        if age in range(20,39+1): return(9) 
        if age in range(40,49+1): return(7) 
        if age in range(50,59+1): return(4) 
        if age in range(60,69+1): return(2)
        if age >= 70: return(1) 

    
def fsbp_pts(sbp,treated):
    if sbp < 120:
        return(0)
    if sbp in range(120,129+1):
        return(3) if treated == 1 else 1
    if sbp in range(130,139+1):
        return(4) if treated == 1 else 2
    if sbp in range(140,159+1):
        return(5) if treated == 1 else 3
    if sbp >= 160:
        return(6) if treated == 1 else 4
    
def fage_pts(age):
    if age in range(45,49+1):
        return(3)
    if age in range(50,54+1):
        return(6)
    if age in range(55,59+1):
        return(8)
    if age in range(60,64+1):
        return(10)
    if age in range(65,69+1):
        return(12)
    if age in range(70,74+1):
        return(14)
    if age >= 75:
        return(16)

def fhdl_pts(hdl):
    if hdl >= 60:
        return(-1)
    if hdl in range(50,59+1):
        return(0)
    if hdl in range(40,49+1):
        return(1)
    if hdl < 40:
        return(2)

Define `check` functions for males

In [6]:
def mtchol_pts(totchol,age):
    if totchol < 160:
        return(0)
    if totchol in range(160,199+1):
        if age in range(20,39+1): return(4)
        if age in range(40,49+1): return(3) 
        if age in range(50,59+1): return(2) 
        if age in range(60,69+1): return(1) 
        if age >= 70: return(0) 
    if totchol in range(200,239+1):
        if age in range(20,39+1): return(7) 
        if age in range(40,49+1): return(5) 
        if age in range(50,59+1): return(3) 
        if age in range(60,69+1): return(1) 
        if age >= 70: return(0) 
    if totchol in range(240,279+1):
        if age in range(20,39+1): return(9) 
        if age in range(40,49+1): return(6) 
        if age in range(50,59+1): return(4) 
        if age in range(60,69+1): return(2) 
        if age >= 70: return(1) 
    if totchol >= 280:
        if age in range(20,39+1): return(11) 
        if age in range(40,49+1): return(8)
        if age in range(50,59+1): return(5)
        if age in range(60,69+1): return(3) 
        if age >= 70:return(1) 


#
 
def msmoke_pts(curr_smq,age):
    if curr_smq == 0:
        return(0)
    if curr_smq == 1:
        if age in range(20,39+1): return(8) 
        if age in range(40,49+1): return(5) 
        if age in range(50,59+1): return(3) 
        if age in range(60,69+1): return(1)
        if age >= 70: return(1)
    
def msbp_pts(sbp,treated):
    if sbp < 120:
        return(0)
    if sbp in range(120,129+1):
        return(1) if treated == 1 else 0
    if sbp in range(130,159+1):
        return(2) if treated == 1 else 1
    if sbp >= 160:
        return(3) if treated == 1 else 2
    
def mage_pts(age):
    if age in range(45,49+1):
        return(3)
    if age in range(50,54+1):
        return(6)
    if age in range(55,59+1):
        return(8)
    if age in range(60,64+1):
        return(10)
    if age in range(65,69+1):
        return(11)
    if age in range(70,74+1):
        return(12)
    if age >= 75:
        return(13)

def mhdl_pts(hdl):
    if hdl >= 60:
        return(-1)
    if hdl in range(50,59+1):
        return(0)
    if hdl in range(40,49+1):
        return(1)
    if hdl < 40:
        return(2)

In [17]:
def get_frs_atp3(sex,age,hdl,totchol,sbp,curr_smq,treated):
    risk_score = 0
    if sex == 1:
        risk_score += mage_pts(age)
        risk_score += mhdl_pts(hdl)
        risk_score += mtchol_pts(totchol,age)
        risk_score += msbp_pts(sbp,treated)
        risk_score += msmoke_pts(curr_smq,age)
    if sex == 2:
        risk_score += fage_pts(age)
        risk_score += fhdl_pts(hdl)
        risk_score += ftchol_pts(totchol,age)
        risk_score += fsbp_pts(sbp,treated)
        risk_score += fsmoke_pts(curr_smq,age)
    return(risk_score)

def get_frs_atp3_nhanesII(x):
    risk_score = 0
    if x["SEX"] == 1:
        risk_score += mage_pts(x["AGEYR_AT_INT"])
        risk_score += mhdl_pts(x["lbdhdd"])
        risk_score += mtchol_pts(x["lbxtc"],x["AGEYR_AT_INT"])
        risk_score += msbp_pts(x["asbp"],x["treated"])
        risk_score += msmoke_pts(x["curr_smq"],x["AGEYR_AT_INT"])
    if x["SEX"] == 2:
        risk_score += fage_pts(x["AGEYR_AT_INT"])
        risk_score += fhdl_pts(x["lbdhdd"])
        risk_score += ftchol_pts(x["lbxtc"],x["AGEYR_AT_INT"])
        risk_score += fsbp_pts(x["asbp"],x["treated"])
        risk_score += fsmoke_pts(x["curr_smq"],x["AGEYR_AT_INT"])
    return(risk_score)


    Need to complete cases of all variable (fields) before running `get_frs` function

In [35]:
from sklearn.datasets import load_iris
data = load_iris()
features = pd.DataFrame(data = data['data'], columns= data ['feature_names'])
features.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [8]:
nhanesII=nhanesII.dropna(subset=["SEX","AGEYR_AT_INT","lbdhdd","lbxtc","asbp","curr_smq","treated"])
# count number of rows
nhanesII.SEX.count()

5218

In [60]:
nhanesII["frs_atp3"]=list(map(get_frs_atp3,nhanesII["SEX"],nhanesII["AGEYR_AT_INT"],nhanesII["lbdhdd"],nhanesII["lbxtc"],nhanesII["asbp"],nhanesII["curr_smq"],nhanesII["treated"])) 
nhanesII["frs_atp3"].head()
nhanesII.frs_atp3.count()

5218

In [142]:
nhanesII.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanesII_frs_atp3.dta")


# NHANES III 

In [116]:
def get_frs_atp3_nhanesIII(x):
    risk_score = 0
    if x["HSSEX"] == 1:
        risk_score += mage_pts(x["HSAGEIR"])
        risk_score += mhdl_pts(x["HDP"])
        risk_score += mtchol_pts(x["TCP"],x["HSAGEIR"])
        risk_score += msbp_pts(x["asbp"],x["treated"])
        risk_score += msmoke_pts(x["curr_smq"],x["HSAGEIR"])
    if x["HSSEX"] == 2:
        risk_score += fage_pts(x["HSAGEIR"])
        risk_score += fhdl_pts(x["HDP"])
        risk_score += ftchol_pts(x["TCP"],x["HSAGEIR"])
        risk_score += fsbp_pts(x["asbp"],x["treated"])
        risk_score += fsmoke_pts(x["curr_smq"],x["HSAGEIR"])
    return(risk_score)

## Phase 1

In [129]:
nhanesIIIP1 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanesIII_P1x_frs_atp3.dta")
nhanesIIIP1.head()

,SEQN,SDPPHASE,WTPFQX6,WTPFEX6,WTPFHX6,WTPFQX1,WTPFEX1,WTPFHX1,WTPFQX2,WTPFEX2,...,TCP,GHP,fpg,_merge,smoke_sts,heaviest_weight_kg,height_meters_sq,maxbmi,curr_smq,former_smq
0,00009,1,18155.150391,19521.359375,19451.830078,36310.308594,39042.718750,38903.671875,NaN,NaN,...,236.0,5.3,111.300003,matched (3),1.0,66.678024,2.461761,27.085501,0.0,0.0
1,00011,1,1166.439941,1248.430054,1245.520020,2332.870117,2496.860107,2491.030029,NaN,NaN,...,260.0,9.1,271.700012,matched (3),1.0,86.182480,2.829124,30.462605,0.0,0.0
2,00048,1,22847.910156,27257.880859,26919.289062,45695.828125,54515.769531,53838.570312,NaN,NaN,...,156.0,5.0,84.900002,matched (3),1.0,117.480331,2.944656,39.896111,0.0,0.0
3,00049,1,3838.909912,6290.859863,5043.859863,7677.819824,12581.719727,10087.719727,NaN,NaN,...,179.0,5.7,83.300003,matched (3),2.0,56.699001,2.286144,24.801151,0.0,0.0
4,00052,1,1292.680054,1401.839966,1398.569946,2585.370117,2803.679932,2797.149902,NaN,NaN,...,244.0,5.9,96.300003,matched (3),2.0,83.914520,3.175524,26.425409,0.0,0.0


In [130]:
print(nhanesIIIP1[["HSSEX","HSAGEIR","HDP","TCP","treated","asbp","curr_smq"]].min())
print(nhanesIIIP1[["HSSEX","HSAGEIR","HDP","TCP","treated","asbp","curr_smq"]].max())
nhanesIIIP1.curr_smq.value_counts()
nhanesIIIP1["curr_smq"][1]==1

HSSEX        1.0
HSAGEIR     45.0
HDP         13.0
TCP         98.0
treated      0.0
asbp        80.0
curr_smq     0.0
dtype: float32
HSSEX         2.0
HSAGEIR      90.0
HDP         191.0
TCP         464.0
treated       1.0
asbp        230.0
curr_smq      1.0
dtype: float32


False

In [131]:
nhanesIIIP1=nhanesIIIP1.dropna(subset=["HSSEX","HSAGEIR","HDP","TCP","treated","asbp","curr_smq"])
# count number of rows
nhanesIIIP1.HSSEX.count()

3827

In [132]:
nhanesIIIP1["frs_atp3"]=nhanesIIIP1.apply(get_frs_atp3_nhanesIII, axis=1)
nhanesIIIP1["frs_atp3"].head()
nhanesIIIP1.frs_atp3.count()

3827

save

In [133]:
nhanesIIIP1.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanesIII_P1x_frs_atp3.dta")

## Phase 2

In [134]:
nhanesIIIP2 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanesIII_P2x_frs_atp3.dta")
nhanesIIIP2.head()

,SEQN,SDPPHASE,WTPFQX6,WTPFEX6,WTPFHX6,WTPFQX1,WTPFEX1,WTPFHX1,WTPFQX2,WTPFEX2,...,TCP,GHP,fpg,_merge,smoke_sts,heaviest_weight_kg,height_meters_sq,maxbmi,curr_smq,former_smq
0,32234,2,3901.040039,4921.580078,4653.689941,NaN,NaN,NaN,7802.080078,9843.160156,...,218.0,5.6,93.199997,matched (3),2.0,84.368111,2.128681,39.633987,0.0,0.0
1,32237,2,17064.070312,22764.519531,19967.550781,NaN,NaN,NaN,34128.128906,45529.031250,...,207.0,4.7,108.699997,matched (3),1.0,78.471413,2.712609,28.928392,0.0,0.0
2,32239,2,20516.560547,21877.720703,21963.820312,NaN,NaN,NaN,41033.121094,43755.441406,...,261.0,5.7,85.699997,matched (3),2.0,97.522278,2.544025,38.333851,0.0,0.0
3,32245,2,1987.650024,2112.399902,2110.250000,NaN,NaN,NaN,3975.310059,4224.790039,...,278.0,6.3,101.199997,matched (3),1.0,88.450439,2.979076,29.690559,0.0,0.0
4,32247,2,1024.560059,1071.859985,1070.800049,NaN,NaN,NaN,2049.120117,2143.709961,...,166.0,4.8,135.199997,matched (3),2.0,68.038803,2.383936,28.540535,0.0,0.0


In [135]:
print(nhanesIIIP2[["HSSEX","HSAGEIR","HDP","TCP","treated","asbp","curr_smq"]].min())
print(nhanesIIIP2[["HSSEX","HSAGEIR","HDP","TCP","treated","asbp","curr_smq"]].max())
nhanesIIIP2.curr_smq.value_counts()
nhanesIIIP2["curr_smq"][1]==1

HSSEX        1.0
HSAGEIR     45.0
HDP         12.0
TCP         59.0
treated      0.0
asbp        81.0
curr_smq     0.0
dtype: float32
HSSEX         2.0
HSAGEIR      90.0
HDP         179.0
TCP         501.0
treated       1.0
asbp        244.0
curr_smq      1.0
dtype: float32


False

In [136]:
nhanesIIIP2=nhanesIIIP2.dropna(subset=["HSSEX","HSAGEIR","HDP","TCP","treated","asbp","curr_smq"])
# count number of rows
nhanesIIIP2.HSSEX.count()

3798

In [137]:
nhanesIIIP2["frs_atp3"]=nhanesIIIP2.apply(get_frs_atp3_nhanesIII, axis=1)
nhanesIIIP2["frs_atp3"].head()
nhanesIIIP2.frs_atp3.count()

3798

save

In [138]:
nhanesIIIP2.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanesIII_P2x_frs_atp3.dta")

# NHANES Continuous

## Process

In [97]:
nhanes1999_2000 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_1999_2000_frs_atp3.dta")
nhanes2001_2002 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_2001_2002_frs_atp3.dta")
nhanes2003_2004 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_2003_2004_frs_atp3.dta")

nhanes2005_2006 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_2005_2006_frs_atp3.dta")

nhanes2007_2008 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_2007_2008_frs_atp3.dta")

nhanes2009_2010 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_2009_2010_frs_atp3.dta")

nhanes2011_2012 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_2011_2012_frs_atp3.dta")

nhanes2013_2014 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_2013_2014_frs_atp3.dta")

nhanes2015_2016 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_2015_2016_frs_atp3.dta")

nhanes2017_2018 = pd.read_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_2017_2018_frs_atp3.dta")

nhanes1999_2000.riagendr.count()

2218

In [98]:
nhanes1999_2000=nhanes1999_2000.dropna(subset=["riagendr","ridageyr","hdl","lbxtc","asbp","curr_smq","treated"])
nhanes2001_2002=nhanes2001_2002.dropna(subset=["riagendr","ridageyr","hdl","lbxtc","asbp","curr_smq","treated"])

nhanes2003_2004=nhanes2003_2004.dropna(subset=["riagendr","ridageyr","hdl","lbxtc","asbp","curr_smq","treated"])

nhanes2005_2006=nhanes2005_2006.dropna(subset=["riagendr","ridageyr","hdl","lbxtc","asbp","curr_smq","treated"])

nhanes2007_2008=nhanes2007_2008.dropna(subset=["riagendr","ridageyr","hdl","lbxtc","asbp","curr_smq","treated"])
nhanes2009_2010=nhanes2009_2010.dropna(subset=["riagendr","ridageyr","hdl","lbxtc","asbp","curr_smq","treated"])


nhanes2011_2012=nhanes2011_2012.dropna(subset=["riagendr","ridageyr","hdl","lbxtc","asbp","curr_smq","treated"])

nhanes2013_2014=nhanes2013_2014.dropna(subset=["riagendr","ridageyr","hdl","lbxtc","asbp","curr_smq","treated"])
nhanes2015_2016=nhanes2015_2016.dropna(subset=["riagendr","ridageyr","hdl","lbxtc","asbp","curr_smq","treated"])
nhanes2017_2018=nhanes2017_2018.dropna(subset=["riagendr","ridageyr","hdl","lbxtc","asbp","curr_smq","treated"])




# count number of rows
nhanes1999_2000.riagendr.count()
nhanes1999_2000.treated.head()
print(nhanes1999_2000[["treated"]].min())

treated    0.0
dtype: float64


In [99]:
def get_frs_atp3_cont(x):
    risk_score = 0
    if x["riagendr"] == 1:
        risk_score += mage_pts(x["ridageyr"])
        risk_score += mhdl_pts(x["hdl"])
        risk_score += mtchol_pts(x["lbxtc"],x["ridageyr"])
        risk_score += msbp_pts(x["asbp"],x["treated"])
        risk_score += msmoke_pts(x["curr_smq"],x["ridageyr"])
    if x["riagendr"] == 2:
        risk_score += fage_pts(x["ridageyr"])
        risk_score += fhdl_pts(x["hdl"])
        risk_score += ftchol_pts(x["lbxtc"],x["ridageyr"])
        risk_score += fsbp_pts(x["asbp"],x["treated"])
        risk_score += fsmoke_pts(x["curr_smq"],x["ridageyr"])
    return(risk_score)


In [100]:
nhanes1999_2000["frs_atp3"]=nhanes1999_2000.apply(get_frs_atp3_cont, axis=1)
nhanes2001_2002["frs_atp3"]=nhanes2001_2002.apply(get_frs_atp3_cont, axis=1)
nhanes2003_2004["frs_atp3"]=nhanes2003_2004.apply(get_frs_atp3_cont, axis=1)
nhanes2005_2006["frs_atp3"]=nhanes2005_2006.apply(get_frs_atp3_cont, axis=1)
nhanes2007_2008["frs_atp3"]=nhanes2007_2008.apply(get_frs_atp3_cont, axis=1)
nhanes2009_2010["frs_atp3"]=nhanes2009_2010.apply(get_frs_atp3_cont, axis=1)
nhanes2011_2012["frs_atp3"]=nhanes2011_2012.apply(get_frs_atp3_cont, axis=1)
nhanes2013_2014["frs_atp3"]=nhanes2013_2014.apply(get_frs_atp3_cont, axis=1)
nhanes2015_2016["frs_atp3"]=nhanes2015_2016.apply(get_frs_atp3_cont, axis=1)
nhanes2017_2018["frs_atp3"]=nhanes2017_2018.apply(get_frs_atp3_cont, axis=1)





nhanes1999_2000["frs_atp3"].head()
nhanes1999_2000.frs_atp3.count()

2218

In [102]:
nhanes1999_2000.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_1999_2000_frs_atp3.dta")
nhanes2001_2002.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_2001_2002_frs_atp3.dta")
nhanes2003_2004.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_2003_2004_frs_atp3.dta")
nhanes2005_2006.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_2005_2006_frs_atp3.dta")
nhanes2007_2008.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_2007_2008_frs_atp3.dta")
nhanes2009_2010.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_2009_2010_frs_atp3.dta")
nhanes2011_2012.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_2011_2012_frs_atp3.dta")
nhanes2013_2014.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_2013_2014_frs_atp3.dta")
nhanes2015_2016.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_2015_2016_frs_atp3.dta")
nhanes2017_2018.to_stata("/Users/bryankim/Documents/NBER/Case Deaton/Data/FRS/ATP3/nhanes_2017_2018_frs_atp3.dta")